In [40]:
%load_ext autoreload
%autoreload 2

In [41]:
from multi_env import make_reversi_vec_env, SelfPlayEnv
import torch as th
from players import RandomPlayer
from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
import numpy as np

In [42]:
board_shape = 8
n_envs = 10
env = make_reversi_vec_env(
    SelfPlayEnv, n_envs=n_envs,
    env_kwargs={
        'board_shape': board_shape,
        'LocalPlayer': RandomPlayer
    }
)

# Modificación de librería para que haga argmax solo sobre las válidas

In [43]:
model = PPO(
    ActorCriticPolicy,
    env,
    verbose=0,
)

In [44]:
model.predict(env.reset())

(array([62, 53, 48, 31, 30, 37, 11, 30,  5, 19]), None)

# Custom ActorCriticPolicy 

https://github.com/DLR-RM/stable-baselines3/blob/master/stable_baselines3/common/policies.py

In [45]:
from boardgame2 import ReversiEnv

In [46]:
env_not_vect = ReversiEnv(board_shape)

In [47]:
(board, player) = env_not_vect.reset()

In [48]:
env_not_vect.get_valid((board, player))

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [49]:
def get_actions_mask(state):
    player = 1
    valid_actions = env_not_vect.get_valid((state, player))
    return valid_actions.reshape(-1)  


In [50]:
get_actions_mask(env.reset()[0][0])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int8)

## Importante: 
### Reemplacé esta sección de la notebook por una implementación en archivos .py para trabajarlo con un IDE.
### El enfoque que seguí fue el de utilizar un 2do canal para contener las máscaras, en vez de llamar a get_actions_mask en forward, predict, etc.
#### * custom_features_extractor.py => Contiene un features extractor que toma el board del canal 0 y hace un flatten, ignorando el canal 1 (mask), para alimentar las nn de actor-critic.
#### * custom_policies.py => Contiene la implementación de CustomActorCritic donde se utiliza el canal 1 para aplicar la máscara de acciones válidas.
#### * multi_env.py => Hice algunas modificaciones minimas a SelfPlayEnv para parametrizar el uso o no del canal de máscara.
#### * reversi_model.py => Contiene una clase de alto nivel para configurar y entrenar el modelo.
#### * main.py => Script de entrada para entrenar por línea de comandos

In [26]:
from reversi_model import CustomReversiModel
from multi_env import SelfPlayEnv
from players import RandomPlayer
import torch as th
import numpy as np

In [10]:
board_shape = 8
n_envs = 8
gamma = 0.99
ent_coef = 0.0
gae_lambda = 0.95
n_epochs = 10

In [11]:
model = CustomReversiModel(board_shape=board_shape, n_envs=n_envs, n_steps=2048,
                           gamma=gamma, ent_coef=ent_coef, gae_lambda=gae_lambda,n_epochs=n_epochs)

Using cuda device


In [12]:
env = SelfPlayEnv(board_shape=board_shape, LocalPlayer=RandomPlayer, mask_channel=True)

In [13]:
obs = env.reset()

In [28]:
obs = np.array([obs]) #Convierto en un batch de 1 (1 env paralelo)

In [39]:
obs[0][0] #Tablero inicial

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, -1,  1,  0,  0,  0],
       [ 0,  0,  0, -1, -1,  0,  0,  0],
       [ 0,  0,  0, -1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]], dtype=int8)

In [33]:
# Testeo de forward
model.model.policy(th.from_numpy(obs).to(model.model.device))

(tensor([44], device='cuda:0'),
 tensor([[-0.1623]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([-4.1593], device='cuda:0', grad_fn=<SqueezeBackward1>))

# Corremos PPO

In [28]:
model.learn(total_timesteps=10e5)

TypeError: can't pickle _thread.lock objects